In [ ]:
# imports
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.utils import text_dataset_from_directory

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        
        return embedded_tokens + embedded_positions
    
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    
    def get_config(self):
        config = super().get_config()
        
        config.update(
            {
                'sequence_length': self.sequence_length,
                'input_dim': self.input_dim,
                'output_dim': self.output_dim
            }
        )
        return config

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        
        self.attention = layers.MultiHeadAttention(key_dim=embed_dim, nun_heads=num_heads)
        
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation='relu'),
                layers.Dense(embed_dim)
            ]
        )
        
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
            
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        
        proj_input = self.layernorm_1(inputs + attention_output)
        
        proj_output = self.dense_proj(proj_input)
        
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self):
        config = super().get_config()
        
        config.update(
            {
                'embed_dim': self.embed_dim,
                'dense_dim': self.dense_dim,
                'num_heads': self.num_heads
            }
        )
        return config

In [4]:
class TrnasformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        
        self.supports_masking = True
        
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        
        self.attention_1 = layers.MultiHeadAttention(key_dim=embed_dim, num_heads=num_heads)
        self.attention_2 = layers.MultiHeadAttention(key_dim=embed_dim, num_heads=num_heads)
        
        self.dense_proj = keras.Sequential(
            [
                layer.Dense(dense_dim, activation='relu'),
                layer.Dense(embed_dim)
            ]
        )
        
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = inputs_shape[0], inputs_shape[1]
        
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        
        mask = tf.cast(i >= j, dtype='int32')
        mask = tf.reshape(mask, (1, sequence_length, sequence_length))
        
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.constant([1, 1], dtype=tf.int32)
            ], axis=0
        )
        
        return tf.tile(mask, mult)
    
    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = get_causal_attention_mask(inputs)
        
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype='int32')
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
            
        attention_output_1 = self.attention_1(
            query=inputs,
            key=inputs,
            value=inputs,
            attention_mask=causal_mask
        )
        
        attention_ouput_1 = self.layernorm_1(inputs + attention_output_1)
        
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            key=encoder_outputs,
            value=encoder_outputs,
            attention_mask=padding_mask
        )
        
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        
        proj_output = self.dense_proj(attention_output_2)
        
        return self.layernorm_3(attention_output_2 + proj_output)
    
    def get_config(self):
        config = super().get_config()
        
        config.update(
            {
                'embed_dim': self.embed_dim,
                'dense_dim': self.dense_dim,
                'num_heads': self.num_heads
            }
        )
        return config

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        
        self.supports_masking = True
        
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        
        self.attention_1 = layers.MultiHeadAttention(key_dim=embed_dim, num_heads=num_heads)
        self.attention_2 = layers.MultiHeadAttention(key_dim=embed_dim, num_heads=num_heads)
        
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation='relu'),
                layers.Dense(embed_dim)
            ]
        )
        
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = inputs_shape[0], inputs_shape[1]
        
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        
        mask = tf.cast(i >= j, dtype='int32')
        mask = tf.reshape(mask, (1, sequence_length, sequence_length))
        
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.constant([1, 1], dtype=tf.int32)
            ], axis=0
        )
        return tf.tile(mask, mult)
    
    

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
with strategy.scope():
    
    encoder_inputs = keras.Input(shape=(None, ), dtype='int64', name='english')
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
    encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
with strategy.scope():
    
    encoder_inputs = keras.Input(shape=(None, ), dtype='int64', name='english')
    x = PositionalEmbedding(sequence_length, vocab_size, embed_dim

In [ ]:
class Node(object):
    def __init__(self, val):
        self.val = val
        self.next = None
        
    def get_data(self):
        return self.val
    
    def set_data(self, val):
        self.val = val
        
    def get_next(self):
        return self.next
    
    def set_next(self, next):
        self.next = next
                
    
        
class LinkedList(object):
    def __init__(self, head=None):
        self.head = head
        self.count = 0
    
    def insert(self, data):
        
        new_node = Node(data)
        new_node.set_next(self.head)
        
        self.head = new_node        
        self.count += 1
        
    def find(self, val):
        item = self.head
        
        while item is not None:
            if item.get_data() == val:
                return item
            else:
                item = item.get_next()
        
        return None
    
    def display(self):
        current = self.head
        while current:
            print(current.val, end=" -> ")
            current = current.next
        print("None")
    
    
    def remove(self, item):
        
        current = self.head
        previous = None
        
        while current is not None:
            if current.get_data() == item:
                break
                
            previous = current
            current = current.get_next()
            
        if current is None: # item not in the list
            raise ValueError(f"{item} is not in the list")
            
        if previous is None: # the item is the head
            self.head = current.get_next()
            self.count -= 1
        else: # just remove the item from the list
            previous.set_next(current.get_next())
            self.count -= 1
    
         
        
            
        
            
    
    def get_count(self):
        return self.count
    
    def is_empty(self):
        return self.head == None